In [21]:
import io
import requests
import pandas as pd
import numpy as np
from scipy.stats import lognorm
from tqdm import tqdm_notebook
from statsmodels.stats.proportion import proportion_confint
from datetime import date, datetime

## Functions

In [22]:
def muTransform(zMedian):
    return np.log(zMedian)

In [23]:
def sigmaTransform(zMean, mu):
    return np.sqrt(2*(np.log(zMean)-mu))

In [24]:
def plnorm(x, mu, sigma):
    shape  = sigma
    loc    = 0
    scale  = np.exp(mu)
    return lognorm.cdf(x, shape, loc, scale)

In [25]:
def hospitalisation_to_death_truncated(x,mu,sigma):
    return plnorm(x + 1, mu, sigma) - plnorm(x, mu, sigma)

def hospitalisation_to_death_truncated_low(x):
    return hospitalisation_to_death_truncated(x,muLow, sigmaLow)

def hospitalisation_to_death_truncated_mid(x):
    return hospitalisation_to_death_truncated(x,muMid, sigmaMid)

def hospitalisation_to_death_truncated_high(x):
    return hospitalisation_to_death_truncated(x,muHigh, sigmaHigh)

## Parameters

In [26]:
# setting the baseline CFR
cCFRBaseline = 1.4
cCFREstimateRange = (1.2, 1.7)
# lower end of the range
zmeanLow = 8.7
zmedianLow = 6.7
# middle of the range
zmeanMid = 13
zmedianMid = 9.1
# upper end of the range
zmeanHigh = 20.9
zmedianHigh = 13.7

In [27]:
muLow=muTransform(zmedianLow)
sigmaLow = sigmaTransform(zmeanLow, muLow)

In [28]:
muMid = muTransform(zmedianMid)
sigmaMid = sigmaTransform(zmeanMid, muMid)

In [29]:
muHigh = muTransform(zmedianHigh)
sigmaHigh = sigmaTransform(zmeanHigh, muHigh)

## Clean Dataset

In [30]:
url="https://opendata.ecdc.europa.eu/covid19/casedistribution/csv"
s=requests.get(url).content
dataset=pd.read_csv(io.StringIO(s.decode('utf-8')))
dataset.tail()

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2018
11147,25/03/2020,25,3,2020,0,0,Zimbabwe,ZW,ZWE,14439018.0
11148,24/03/2020,24,3,2020,0,1,Zimbabwe,ZW,ZWE,14439018.0
11149,23/03/2020,23,3,2020,0,0,Zimbabwe,ZW,ZWE,14439018.0
11150,22/03/2020,22,3,2020,1,0,Zimbabwe,ZW,ZWE,14439018.0
11151,21/03/2020,21,3,2020,1,0,Zimbabwe,ZW,ZWE,14439018.0


In [31]:
dataset.rename(columns = {
    "dateRep": "date",
    "cases": "new_cases",
    "deaths": "new_deaths",
    "countriesAndTerritories": "country"
},inplace=True)
allTogetherClean = dataset[['date', 'country', 'new_cases', 'new_deaths']]

In [32]:
## Exclude some countries
exclude_coutries = ['Canada','Cases_on_an_international_conveyance_Japan']
allTogetherClean = allTogetherClean[~allTogetherClean.country.isin(exclude_coutries)]
allTogetherClean.tail()

,date,country,new_cases,new_deaths
11147,25/03/2020,Zimbabwe,0,0
11148,24/03/2020,Zimbabwe,0,1
11149,23/03/2020,Zimbabwe,0,0
11150,22/03/2020,Zimbabwe,1,0
11151,21/03/2020,Zimbabwe,1,0


In [33]:
## Remove lower data points
threshold = 10
list_filtered_countried = allTogetherClean.groupby('country').filter(lambda x: x['new_deaths'].sum()>threshold)['country'].unique()

In [34]:
allTogetherClean = allTogetherClean[allTogetherClean.country.isin(list_filtered_countried)].reset_index(drop=True)
allTogetherClean.tail()

,date,country,new_cases,new_deaths
6531,04/01/2020,United_States_of_America,0,0
6532,03/01/2020,United_States_of_America,0,0
6533,02/01/2020,United_States_of_America,0,0
6534,01/01/2020,United_States_of_America,0,0
6535,31/12/2019,United_States_of_America,0,0


## Calculate UnderReporting

$$u_{t}=\frac{\sum_{j=0}^{t}c_{t-j}f_{j}}{c_{t}}$$

where:  
$u_{t}$ = underestimation of the proportion of cases with known outcomes  
$c_{t}$ = daily case incidence at time t  
$f_{t}$ = proportion of cases with delay of t between confirmation and death

In [35]:
def calculate_underestimate(country, delay_func):
    df = allTogetherClean[allTogetherClean.country==country].iloc[::-1].reset_index(drop=True)
    cumulative_known_t = 0
    for ii in range(0,len(df)):
        #print("ii",ii)
        known_i = 0
        for jj in range(0,ii+1):
            #print("jj",jj)
            known_jj = df['new_cases'].loc[ii-jj]*delay_func(jj)
            known_i = known_i + known_jj
        cumulative_known_t = cumulative_known_t + known_i
        #print("-"*30)
    cum_known_t = round(cumulative_known_t)
    # naive CFR value
    nCFR = df['new_deaths'].sum()/df['new_cases'].sum()
    # corrected CFR estimator
    cCFR = df['new_deaths'].sum()/cum_known_t
    total_deaths = df['new_deaths'].sum()
    total_cases = df['new_cases'].sum()
    nCFR_UQ, nCFR_LQ =  proportion_confint(total_deaths, total_cases)
    cCFR_UQ, cCFR_LQ =  proportion_confint(total_deaths, cum_known_t)
    return nCFR, cCFR, total_deaths, cum_known_t, total_cases, round(nCFR_UQ,8), round(nCFR_LQ,8), round(cCFR_UQ,8), round(cCFR_LQ,8)

In [36]:
calculate_underestimate("Albania", hospitalisation_to_death_truncated_low)

(0.05060728744939271,
 0.06720430107526881,
 25,
 372.0,
 494,
 0.03127808,
 0.0699365,
 0.04176129,
 0.09264731)

In [37]:
calculate_underestimate("Afghanistan", hospitalisation_to_death_truncated_low)

(0.03188775510204082,
 0.05868544600938967,
 25,
 426.0,
 784,
 0.01958889,
 0.04418662,
 0.03636639,
 0.08100451)

In [38]:
calculate_underestimate("Argentina", hospitalisation_to_death_truncated_low)

(0.04481907894736842,
 0.0632617527568195,
 109,
 1723.0,
 2432,
 0.03659588,
 0.05304227,
 0.05176738,
 0.07475612)

In [39]:
calculate_underestimate("Brazil", hospitalisation_to_death_truncated_low)

(0.06129943502824859,
 0.111104,
 1736,
 15625.0,
 28320,
 0.05850565,
 0.06409322,
 0.10617648,
 0.11603152)

In [40]:
def return_complete_df(dataframe, delay_func):
    all_countries = dataframe['country'].unique()
    new_df = pd.DataFrame(columns = [
        'country',
        'nCFR', 
        'cCFR', 
        'total_deaths', 
        'cum_known_t', 
        'total_cases',
        'nCFR_UQ',
        'nCFR_LQ',
        'cCFR_UQ',
        'cCFR_LQ',
        'underreporting_estimate',
        'lower',
        'upper',
        'quantile25',
        'quantile75'
    ])
    
    for c in tqdm_notebook(all_countries):
        nCFR, cCFR, total_deaths, cum_known_t, total_cases, nCFR_UQ, nCFR_LQ,cCFR_UQ, cCFR_LQ = calculate_underestimate(c,delay_func)
        quantile25, quantile75 = proportion_confint(total_deaths, cum_known_t, alpha = 0.5)
        new_df = new_df.append({'country':c,
                       'nCFR':nCFR,
                       'cCFR': cCFR,
                       'total_deaths': total_deaths,
                       'cum_known_t': int(cum_known_t),
                       'total_cases': total_cases,
                       'nCFR_UQ': nCFR_UQ,
                       'nCFR_LQ': nCFR_LQ,
                       'cCFR_UQ': cCFR_UQ,
                       'cCFR_LQ': cCFR_LQ,
                       'underreporting_estimate': cCFRBaseline / (100*cCFR),
                       'lower': cCFREstimateRange[0]/ (100 * cCFR_UQ),
                       'upper': cCFREstimateRange[1]/ (100 * cCFR_LQ),
                       'quantile25': quantile25,
                       'quantile75': quantile75            
                      }, ignore_index=True)
    return new_df

In [41]:
allTogetherLow = return_complete_df(allTogetherClean, hospitalisation_to_death_truncated_low)
allTogetherMid = return_complete_df(allTogetherClean, hospitalisation_to_death_truncated_mid)
allTogetherHigh = return_complete_df(allTogetherClean, hospitalisation_to_death_truncated_high)

## Saving Output

In [53]:
!ls -lah ../output/

total 240
drwxr-xr-x  8 rsilvei  813930708   256B Apr 16 22:14 .
drwxr-xr-x  7 rsilvei  813930708   224B Apr 16 11:36 ..
-rw-r--r--  1 rsilvei  813930708    17K Apr 16 11:44 high.csv
-rw-r--r--  1 rsilvei  813930708    17K Apr 16 22:14 high_2020-04-16-22h.csv
-rw-r--r--  1 rsilvei  813930708    17K Apr 16 11:44 low.csv
-rw-r--r--  1 rsilvei  813930708    17K Apr 16 22:14 low_2020-04-16-22h.csv
-rw-r--r--  1 rsilvei  813930708    17K Apr 16 11:44 mid.csv
-rw-r--r--  1 rsilvei  813930708    17K Apr 16 22:14 mid_2020-04-16-22h.csv


In [43]:
date_tag = datetime.now().strftime("%Y-%m-%d-%Hh")

In [44]:
allTogetherLow.to_csv("../output/low_{}.csv".format(date_tag))
allTogetherMid.to_csv("../output/mid_{}.csv".format(date_tag))
allTogetherHigh.to_csv("../output/high_{}.csv".format(date_tag))

In [54]:
!ls -lah ../output/

total 240
drwxr-xr-x  8 rsilvei  813930708   256B Apr 16 22:14 .
drwxr-xr-x  7 rsilvei  813930708   224B Apr 16 11:36 ..
-rw-r--r--  1 rsilvei  813930708    17K Apr 16 11:44 high.csv
-rw-r--r--  1 rsilvei  813930708    17K Apr 16 22:14 high_2020-04-16-22h.csv
-rw-r--r--  1 rsilvei  813930708    17K Apr 16 11:44 low.csv
-rw-r--r--  1 rsilvei  813930708    17K Apr 16 22:14 low_2020-04-16-22h.csv
-rw-r--r--  1 rsilvei  813930708    17K Apr 16 11:44 mid.csv
-rw-r--r--  1 rsilvei  813930708    17K Apr 16 22:14 mid_2020-04-16-22h.csv


## Function for Individual Country

In [46]:
def underreporting_estimate_country(country):
    nCFR_low, cCFR_low, _, _, _, _, _,_, _ = calculate_underestimate(country,hospitalisation_to_death_truncated_low)
    nCFR_mid, cCFR_mid, _, _, _, _, _,_, _ = calculate_underestimate(country,hospitalisation_to_death_truncated_mid)
    nCFR_high, cCFR_high, total_deaths, cum_known_t, total_cases, _, _,_, _ = calculate_underestimate(country,hospitalisation_to_death_truncated_high)
    
    underreporting_estimate_low = cCFRBaseline / (100*cCFR_low)
    underreporting_estimate_mid = cCFRBaseline / (100*cCFR_mid)
    underreporting_estimate_high =  cCFRBaseline / (100*cCFR_high)
    print(nCFR_low, nCFR_mid, nCFR_high)
    return { 'naive_CFR_': "{:.2f}%".format(nCFR_mid*100),
             'underreporting_low':   "{:.2f}%".format(underreporting_estimate_low*100),
             'underreporting_mid':   "{:.2f}%".format(underreporting_estimate_mid*100),
             'underreporting_high':  "{:.2f}%".format(underreporting_estimate_high*100)
           }

In [47]:
underreporting_estimate_country("Brazil")

0.06129943502824859 0.06129943502824859 0.06129943502824859


{'naive_CFR_': '6.13%',
 'underreporting_low': '12.60%',
 'underreporting_mid': '10.18%',
 'underreporting_high': '7.33%'}

In [48]:
underreporting_estimate_country("United_States_of_America")

0.04843949323394782 0.04843949323394782 0.04843949323394782


{'naive_CFR_': '4.84%',
 'underreporting_low': '18.70%',
 'underreporting_mid': '15.33%',
 'underreporting_high': '11.27%'}

In [50]:
underreporting_estimate_country("Lebanon")

0.031914893617021274 0.031914893617021274 0.031914893617021274


{'naive_CFR_': '3.19%',
 'underreporting_low': '36.60%',
 'underreporting_mid': '32.47%',
 'underreporting_high': '26.47%'}

In [51]:
underreporting_estimate_country("Germany")

0.027359141433499424 0.027359141433499424 0.027359141433499424


{'naive_CFR_': '2.74%',
 'underreporting_low': '39.88%',
 'underreporting_mid': '34.10%',
 'underreporting_high': '26.51%'}

In [52]:
underreporting_estimate_country("Switzerland")

0.03694562575941677 0.03694562575941677 0.03694562575941677


{'naive_CFR_': '3.69%',
 'underreporting_low': '30.97%',
 'underreporting_mid': '26.93%',
 'underreporting_high': '21.37%'}

# Sources
1. https://cmmid.github.io/topics/covid19/severity/global_cfr_estimates.html